# Multi-Model Ensemble (MLP + XGBoost + RF + LGBM)

**CRITICAL INSIGHT**: After 49 experiments, ALL model families follow the SAME CV-LB relationship.
- LB = 4.23×CV + 0.0533 (R²=0.981)
- Intercept (0.0533) > Target (0.0347) → Current approach CANNOT reach target

**Hypothesis**: Different model families might have different biases that cancel out when combined.

**Implementation**:
- Combine MLP + XGBoost + RandomForest + LightGBM (like the "mixall" kernel)
- Each model might make different errors on novel solvents
- The ensemble might have a different CV-LB relationship
- Use Spange + DRFP + ACS PCA + Arrhenius kinetics features

**Why this might work**:
- The "mixall" kernel uses this approach and claims good CV/LB
- Different model families capture different aspects of the data
- Combining them might reduce the intercept

In [1]:
# Standard imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor
import tqdm
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.set_default_dtype(torch.double)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [2]:
# Data loading functions
DATA_PATH = '/home/data'

INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_FULL_SOLVENT = ["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

print('Data loading functions defined')

Data loading functions defined


In [3]:
# Load feature lookups
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
DRFP_DF = pd.read_csv(f'{DATA_PATH}/drfps_catechol_lookup.csv', index_col=0)
ACS_PCA_DF = pd.read_csv(f'{DATA_PATH}/acs_pca_descriptors_lookup.csv', index_col=0)

# Filter DRFP to high-variance columns
drfp_variance = DRFP_DF.var()
nonzero_variance_cols = drfp_variance[drfp_variance > 0].index.tolist()
DRFP_FILTERED = DRFP_DF[nonzero_variance_cols]

print(f'Spange: {SPANGE_DF.shape}, DRFP filtered: {DRFP_FILTERED.shape}, ACS PCA: {ACS_PCA_DF.shape}')

Spange: (26, 13), DRFP filtered: (24, 122), ACS PCA: (24, 5)


In [4]:
# Combined Featurizer (Spange + DRFP + ACS PCA + Arrhenius kinetics)
class CombinedFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.drfp_df = DRFP_FILTERED
        self.acs_pca_df = ACS_PCA_DF
        self.feats_dim = 2 + 3 + self.spange_df.shape[1] + self.drfp_df.shape[1] + self.acs_pca_df.shape[1]

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            A_drfp = self.drfp_df.loc[X["SOLVENT A NAME"]].values
            B_drfp = self.drfp_df.loc[X["SOLVENT B NAME"]].values
            A_acs = self.acs_pca_df.loc[X["SOLVENT A NAME"]].values
            B_acs = self.acs_pca_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
                X_drfp = B_drfp * (1 - (1-pct)) + A_drfp * (1-pct)
                X_acs = B_acs * (1 - (1-pct)) + A_acs * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
                X_drfp = A_drfp * (1 - pct) + B_drfp * pct
                X_acs = A_acs * (1 - pct) + B_acs * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
            X_drfp = self.drfp_df.loc[X["SOLVENT NAME"]].values
            X_acs = self.acs_pca_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange, X_drfp, X_acs])

print(f'Combined feature dimension: {CombinedFeaturizer().feats_dim}')

Combined feature dimension: 145


In [5]:
# Simple MLP for ensemble
class SimpleMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, output_dim=3, dropout=0.2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.LayerNorm(hidden_dim // 2),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim // 2, output_dim)
        )
        
    def forward(self, x):
        return self.net(x)

print('SimpleMLP defined')

SimpleMLP defined


In [6]:
# Multi-Model Ensemble (MLP + XGBoost + RF + LGBM)
class MultiModelEnsemble:
    """Multi-model ensemble combining MLP + XGBoost + RandomForest + LightGBM.
    
    Hypothesis: Different model families might have different biases that cancel out.
    """
    def __init__(self, data='single', weights=[0.4, 0.2, 0.2, 0.2]):
        self.data_type = data
        self.weights = weights  # [MLP, XGB, RF, LGBM]
        self.featurizer = CombinedFeaturizer(mixed=(data == 'full'))
        self.scaler = StandardScaler()
        self.y_scaler = StandardScaler()
        
        # Models
        self.mlp = SimpleMLP(self.featurizer.feats_dim, hidden_dim=64, output_dim=3, dropout=0.2).to(device)
        self.xgb_models = []
        self.rf_models = []
        self.lgbm_models = []
        
    def train_model(self, X, Y):
        X_feat = self.featurizer.featurize(X)
        X_scaled = self.scaler.fit_transform(X_feat)
        Y_np = Y.values
        Y_scaled = self.y_scaler.fit_transform(Y_np)
        
        # Train MLP
        self._train_mlp(X_scaled, Y_scaled)
        
        # Train XGBoost (one per target)
        self.xgb_models = []
        for i in range(3):
            model = xgb.XGBRegressor(
                n_estimators=100, learning_rate=0.05, max_depth=4,
                reg_alpha=0.1, reg_lambda=0.1, random_state=42, verbosity=0
            )
            model.fit(X_scaled, Y_scaled[:, i])
            self.xgb_models.append(model)
        
        # Train RandomForest (one per target)
        self.rf_models = []
        for i in range(3):
            model = RandomForestRegressor(
                n_estimators=100, max_depth=6, min_samples_leaf=5,
                random_state=42, n_jobs=-1
            )
            model.fit(X_scaled, Y_scaled[:, i])
            self.rf_models.append(model)
        
        # Train LightGBM (one per target)
        self.lgbm_models = []
        for i in range(3):
            model = lgb.LGBMRegressor(
                n_estimators=100, learning_rate=0.05, max_depth=4,
                num_leaves=15, min_child_samples=10, reg_alpha=0.1, reg_lambda=0.1,
                random_state=42, verbose=-1
            )
            model.fit(X_scaled, Y_scaled[:, i])
            self.lgbm_models.append(model)
    
    def _train_mlp(self, X_scaled, Y_scaled):
        X_tensor = torch.tensor(X_scaled, dtype=torch.double).to(device)
        Y_tensor = torch.tensor(Y_scaled, dtype=torch.double).to(device)
        
        dataset = TensorDataset(X_tensor, Y_tensor)
        loader = DataLoader(dataset, batch_size=32, shuffle=True)
        
        optimizer = torch.optim.AdamW(self.mlp.parameters(), lr=1e-3, weight_decay=1e-4)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100)
        
        target_weights = torch.tensor([1.0, 1.0, 2.0], dtype=torch.double).to(device)
        
        self.mlp.train()
        for epoch in range(100):
            for X_batch, Y_batch in loader:
                optimizer.zero_grad()
                pred = self.mlp(X_batch)
                loss = ((pred - Y_batch) ** 2 * target_weights).mean()
                loss.backward()
                optimizer.step()
            scheduler.step()
    
    def predict(self, X):
        X_feat = self.featurizer.featurize(X)
        X_scaled = self.scaler.transform(X_feat)
        
        # MLP prediction
        self.mlp.eval()
        with torch.no_grad():
            X_tensor = torch.tensor(X_scaled, dtype=torch.double).to(device)
            pred_mlp = self.mlp(X_tensor).cpu().numpy()
        
        # XGBoost prediction
        pred_xgb = np.column_stack([m.predict(X_scaled) for m in self.xgb_models])
        
        # RandomForest prediction
        pred_rf = np.column_stack([m.predict(X_scaled) for m in self.rf_models])
        
        # LightGBM prediction
        pred_lgbm = np.column_stack([m.predict(X_scaled) for m in self.lgbm_models])
        
        # Weighted average (in scaled space)
        pred_scaled = (
            self.weights[0] * pred_mlp +
            self.weights[1] * pred_xgb +
            self.weights[2] * pred_rf +
            self.weights[3] * pred_lgbm
        )
        
        # Inverse transform
        pred = self.y_scaler.inverse_transform(pred_scaled)
        pred = np.clip(pred, 0, 1)
        
        return torch.tensor(pred)
    
    def predict_with_tta(self, X):
        if self.data_type == 'single':
            return self.predict(X)
        
        pred1 = self.predict(X)
        
        X_flip = X.copy()
        X_flip["SOLVENT A NAME"] = X["SOLVENT B NAME"]
        X_flip["SOLVENT B NAME"] = X["SOLVENT A NAME"]
        X_flip["SolventB%"] = 1 - X["SolventB%"]
        
        pred2 = self.predict(X_flip)
        
        return (pred1 + pred2) / 2

print('MultiModelEnsemble defined with MLP(0.4) + XGB(0.2) + RF(0.2) + LGBM(0.2)')

MultiModelEnsemble defined with MLP(0.4) + XGB(0.2) + RF(0.2) + LGBM(0.2)


In [7]:
# Test the model
X_single, Y_single = load_data("single_solvent")
print(f"Single solvent data: {X_single.shape[0]} samples, {len(X_single['SOLVENT NAME'].unique())} unique solvents")

X_full, Y_full = load_data("full")
ramps = X_full[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
print(f"Full data: {X_full.shape[0]} samples, {len(ramps)} unique ramps")

Single solvent data: 656 samples, 24 unique solvents
Full data: 1227 samples, 13 unique ramps


In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = MultiModelEnsemble(data='single')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_single_solvent = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = MultiModelEnsemble(data='full')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict_with_tta(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
# Calculate CV score (for verification only - NOT part of submission)
X_single, Y_single = load_data("single_solvent")
X_full, Y_full = load_data("full")

# Get actuals in same order as predictions
actuals_single = []
for solvent in sorted(X_single["SOLVENT NAME"].unique()):
    mask = X_single["SOLVENT NAME"] == solvent
    actuals_single.append(Y_single[mask].values)
actuals_single = np.vstack(actuals_single)

actuals_full = []
ramps = X_full[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
for _, row in ramps.iterrows():
    mask = (X_full["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X_full["SOLVENT B NAME"] == row["SOLVENT B NAME"])
    actuals_full.append(Y_full[mask].values)
actuals_full = np.vstack(actuals_full)

# Get predictions
preds_single = submission_single_solvent[['target_1', 'target_2', 'target_3']].values
preds_full = submission_full_data[['target_1', 'target_2', 'target_3']].values

# Calculate MSE
mse_single = np.mean((actuals_single - preds_single) ** 2)
mse_full = np.mean((actuals_full - preds_full) ** 2)
n_single = len(actuals_single)
n_full = len(actuals_full)
overall_mse = (mse_single * n_single + mse_full * n_full) / (n_single + n_full)

print(f'\n=== MULTI-MODEL ENSEMBLE CV SCORE ===')
print(f'Single Solvent MSE: {mse_single:.6f} (n={n_single})')
print(f'Full Data MSE: {mse_full:.6f} (n={n_full})')
print(f'Overall MSE: {overall_mse:.6f}')

print(f'\n=== COMPARISON ===')
print(f'exp_032 (best CV, GP 0.15 + MLP 0.55 + LGBM 0.3): CV 0.008194')
print(f'exp_050 (MULTI-MODEL ENSEMBLE): CV {overall_mse:.6f}')

if overall_mse < 0.008194:
    improvement = (0.008194 - overall_mse) / 0.008194 * 100
    print(f'\n✓ IMPROVEMENT: {improvement:.2f}% better than exp_032!')
else:
    degradation = (overall_mse - 0.008194) / 0.008194 * 100
    print(f'\n✗ WORSE: {degradation:.2f}% worse than exp_032')
    
# Estimate LB using old relationship
estimated_lb = 4.23 * overall_mse + 0.0533
print(f'\nEstimated LB (using old relationship): {estimated_lb:.4f}')
print(f'Best LB so far: 0.0877')
print(f'Target: 0.0347')
print(f'\nKey question: Does Multi-Model Ensemble have a DIFFERENT CV-LB relationship?')
print(f'If it has lower intercept, we might be able to reach the target.')